Textural Parameter calculations

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
from skimage.feature import greycomatrix
from scipy.spatial.distance import pdist, squareform
from scipy.ndimage import label


# Define a threshold for binary conversion (if needed)
some_threshold = 0.5


def calculate_fractal_dimension(image, scale_factor):

    binary_image = (image > some_threshold).astype(int)
    labeled_image, num_features = label(binary_image)
    fractal_dimension = np.log(num_features) / np.log(1.0 / scale_factor)
    return fractal_dimension

# Entropy Calculation
def calculate_entropy(image):
    entropy = scipy.stats.entropy(image)
    return entropy

# Lacunarity Calculation
def calculate_lacunarity(image):
    mean = np.mean(image)
    variance = np.var(image)
    lacunarity = variance / (mean**2)
    return lacunarity

# Semivariogram Calculation
def calculate_semivariogram(image):
    image = img_as_ubyte(image)
    glcm = greycomatrix(image, [1], [0], symmetric=True, normed=True)
    distances = squareform(pdist(np.argwhere(image > some_threshold)))
    semivariogram = np.var(glcm) * np.mean((distances - np.mean(distances))**2)
    return semivariogram

In [ ]:
from google.colab import drive
import zipfile

# Mount Google Drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Calculating the values for each image from the dataset

In [ ]:
import os
import numpy as np
from skimage import io, img_as_ubyte, feature
import pandas as pd
from zipfile import ZipFile
from google.colab import drive


# Specify the path to the zip file containing your dataset in Google Drive
zip_file_path = '/content/drive/MyDrive/data/gtos-mobile.zip'

folders_to_process = ['brick', 'cement', 'grass', 'moss', 'pebble', 'sand', 'small_limestone', 'soil' ,'stone_asphalt', 'stone_brick', 'stone_cement']

# Initialize lists to store results
file_names = []
fractal_dimensions = []
lacunarity_values = []
semivariogram_values = []
entropy_values = []  # New list for entropy
material_labels = []

# Open the zip file and loop through its contents
with ZipFile(zip_file_path, 'r') as zip_file:
    for folder_name in folders_to_process:
        folder_prefix = 'gtos-mobile/train/'+ folder_name + '/'  # Adjust this based on your folder structure
        for entry in zip_file.infolist():
            if entry.filename.startswith(folder_prefix) and (entry.filename.endswith('.jpg') or entry.filename.endswith('.png')):
                # Read the image directly from the zip file
                with zip_file.open(entry.filename) as file:
                    image = io.imread(file, as_gray=True)
                    image = img_as_ubyte(image)
                    smaller_image = (image - np.min(image)) / (np.max(image) - np.min(image))

                    fractal_dim = calculate_fractal_dimension(smaller_image, scale_factor=2)  # Replace with your fractal dimension calculation function
                    lacunarity = calculate_lacunarity(smaller_image)  # Replace with your lacunarity calculation function
                    semivariogram = calculate_semivariogram(image)  # Replace with your semivariogram calculation function
                    # entropy = calculate_entropy(smaller_image)  # Replace with your entropy calculation function

                    # Append results to lists
                    file_names.append(entry.filename)
                    fractal_dimensions.append(fractal_dim)
                    lacunarity_values.append(lacunarity)
                    semivariogram_values.append(semivariogram)
                    # entropy_values.append(entropy)  # Add entropy value

                    # Extract material label from the filename or directory structure
                    material_label = folder_name  # Assuming folder name is the material label
                    material_labels.append(material_label)

# Create a DataFrame
data = {
    'filename': file_names,
    'Fractal_dimension': fractal_dimensions,
    'Lacunarity': lacunarity_values,
    'Semivariogram': semivariogram_values,
    # 'Entropy': entropy_values,  # Include entropy in the DataFrame
    'Material_Label': material_labels
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = '/content/drive/MyDrive/data/texture_features_tree.csv'
df.to_csv(csv_file_path, index=False)

print("Texture features saved to texture_features.csv")


/usr/local/lib/python3.10/dist-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')


In [ ]:
import pandas as pd

d = pd.read_csv("/content/drive/MyDrive/data/terrain_roughness_classification.csv")

d.describe()

,Fractal_dimension,Lacunarity
count,38646.000000,38646.000000
mean,-7.920970,0.121026
std,2.088876,0.099649
min,-12.136671,0.000511
25%,-9.379378,0.046942
50%,-8.179909,0.093242
75%,-6.820179,0.165296
max,-0.000000,1.204755


Adding the labels for Roughness estimation based on the Statistics of the calculated parameters

In [ ]:
df = pd.DataFrame(d)

# Calculate the 25th and 75th percentiles (Q1 and Q3) for both fractal dimension and lacunarity
Q1_fractal = np.percentile(df['Fractal_dimension'], 25)
Q3_fractal = np.percentile(df['Fractal_dimension'], 75)
Q1_lacunarity = np.percentile(df['Lacunarity'], 25)
Q3_lacunarity = np.percentile(df['Lacunarity'], 75)

# Define thresholds for terrain roughness based on percentiles
threshold_smooth_fractal = Q1_fractal
threshold_rough_fractal = Q3_fractal
threshold_smooth_lacunarity = Q1_lacunarity
threshold_rough_lacunarity = Q3_lacunarity

# Classify terrain roughness based on thresholds
def classify_terrain_roughness(row):
    if (
        row['Fractal_dimension'] <= threshold_smooth_fractal
        and row['Lacunarity'] <= threshold_smooth_lacunarity
    ):
        return "Relatively Smooth Terrain"
    elif (
        row['Fractal_dimension'] >= threshold_rough_fractal
        or row['Lacunarity'] >= threshold_rough_lacunarity
    ):
        return "Rough Terrain"
    else:
        return "Moderate rough"

# Apply the classification function and add a new column to the DataFrame
df['Terrain_Roughness'] = df.apply(classify_terrain_roughness, axis=1)


Training the machine learning model

In [ ]:
file_path = 'terrain_roughness_classification.csv'

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data/terrain_roughness_classification.csv")

In [ ]:
data.head()

,filename,Fractal_dimension,Lacunarity,Material_Label,Terrain_Roughness
0,gtos-mobile/train/brick/512_sample154_04_i13h.jpg,-6.918863,0.086210,brick,Moderate Terrain Roughness
1,gtos-mobile/train/brick/512_sample154_04_i09l.jpg,-7.330917,0.035026,brick,Moderate Terrain Roughness
2,gtos-mobile/train/brick/512_sample154_04_i11l.jpg,-7.357552,0.071256,brick,Moderate Terrain Roughness
3,gtos-mobile/train/brick/384_sample154_04_i12l.jpg,-9.967226,0.059995,brick,Moderate Terrain Roughness
4,gtos-mobile/train/brick/512_sample154_04_i14h.jpg,-5.584963,0.065416,brick,Rough Terrain


In [ ]:
df_1  = pd.DataFrame(data)
shuffled_df = df_1.sample(frac=1, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# Split the data into features (X) and labels (y) for terrain roughness and material label
X = shuffled_df[['Fractal_dimension', 'Lacunarity']]
y_terrain_roughness = shuffled_df['Terrain_Roughness']
y_material_label = shuffled_df['Material_Label']

# Split the data into training and testing sets (you can adjust the test_size as needed)
X_train, X_test, y_train_terrain, y_test_terrain, y_train_material, y_test_material = train_test_split(
    X, y_terrain_roughness, y_material_label, test_size=0.2, random_state=42
)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training data and transform both the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
import joblib
# Initialize and train a Random Forest Classifier for terrain roughness
clf_terrain = RandomForestClassifier(n_estimators=100, random_state=42)
clf_terrain.fit(X_train_scaled, y_train_terrain)



# Assuming 'clf_terrain' is your trained Random Forest Classifier
joblib.dump(clf_terrain, 'terrain_model.joblib')
#

['terrain_model.joblib']

In [ ]:
#Make predictions on terrain roughness for the test set
y_pred_terrain = clf_terrain.predict(X_test_scaled)



In [ ]:
# Evaluate the terrain roughness classifier
terrain_accuracy = accuracy_score(y_test_terrain, y_pred_terrain)
print("Terrain Roughness Classifier Accuracy:", terrain_accuracy)
print("\nClassification Report for Terrain Roughness:\n", classification_report(y_test_terrain, y_pred_terrain))



Terrain Roughness Classifier Accuracy: 1.0

Classification Report for Terrain Roughness:
                             precision    recall  f1-score   support

Moderate Terrain Roughness       1.00      1.00      1.00      3762
 Relatively Smooth Terrain       1.00      1.00      1.00       577
             Rough Terrain       1.00      1.00      1.00      3391

                  accuracy                           1.00      7730
                 macro avg       1.00      1.00      1.00      7730
              weighted avg       1.00      1.00      1.00      7730



In [ ]:
# Initialize and train a Random Forest Classifier for material label
clf_material = RandomForestClassifier(n_estimators=100, random_state=42)
clf_material.fit(X_train_scaled, y_train_material)

# Make predictions on material label for the test set
y_pred_material = clf_material.predict(X_test_scaled)

# Evaluate the material label classifier
material_accuracy = accuracy_score(y_test_material, y_pred_material)
print("Material Label Classifier Accuracy:", material_accuracy)
print("\nClassification Report for Material Label:\n", classification_report(y_test_material, y_pred_material))

joblib.dump(clf_material, 'material_model.joblib')

Material Label Classifier Accuracy: 0.291849935316947

Classification Report for Material Label:
                  precision    recall  f1-score   support

          brick       0.30      0.29      0.29       966
         cement       0.46      0.50      0.48      1535
          grass       0.24      0.27      0.26       597
           moss       0.20      0.20      0.20       521
         pebble       0.31      0.35      0.33       645
           sand       0.21      0.17      0.19       488
small_limestone       0.29      0.31      0.30       791
           soil       0.22      0.21      0.21       858
  stone_asphalt       0.11      0.08      0.09       413
    stone_brick       0.28      0.28      0.28       414
   stone_cement       0.14      0.12      0.13       502

       accuracy                           0.29      7730
      macro avg       0.25      0.25      0.25      7730
   weighted avg       0.28      0.29      0.29      7730



['material_model.joblib']

Testing on random images

In [ ]:
import cv2
from skimage import io, img_as_ubyte, feature

# Load and preprocess the single image (replace 'image_path.jpg' with the actual image path)
image_path = '/content/photo1695488130.jpeg'
features = []



# Extract relevant features (fractal dimension and lacunarity) from the image
image = io.imread(image_path, as_gray=True)
image = img_as_ubyte(image)
smaller_image = (image - np.min(image)) / (np.max(image) - np.min(image))

def extract_features(image):
    # Implement your feature extraction method here
    # Example: Calculate fractal dimension and lacunarity from the image
    fractal_dimension = calculate_fractal_dimension(image, scale_factor = 2)
    lacunarity = calculate_lacunarity(image)

    # Return the features as a dictionary
    features = {
        'Fractal_dimension': fractal_dimension,
        'Lacunarity': lacunarity
    }

    return features

# Extract features from the image
image_features = extract_features(smaller_image)

# Create a DataFrame from the extracted features
df_in = pd.DataFrame([image_features])

# Scale the features using the same scaler used during model training
# Initialize the StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

# Use the scaler to transform the extracted features
scaled_features = scaler.transform(df_in)  # Replace 'features' with the extracted features

# Initialize the trained Random Forest Classifier for terrain roughness
# clf_terrain = RandomForestClassifier(n_estimators=100, random_state=42)

# Load the trained model weights (replace 'model_weights.pkl' with your trained model's file)
loaded_model = joblib.load('terrain_model.joblib')
loaded_model_1 = joblib.load('material_model.joblib')


# Make predictions on the single image
predicted_terrain = loaded_model.predict(scaled_features)
predicted_label = loaded_model_1.predict(scaled_features)

# Print the predicted terrain roughness
print("Predicted Terrain Roughness:", predicted_terrain)
print("Predicted Terrain type:", predicted_label)

Predicted Terrain Roughness: ['Moderate Terrain Roughness']
Predicted Terrain type: ['small_limestone']
